In [1]:
import warnings
warnings.filterwarnings('ignore')
import os
os.environ['PROJ_LIB'] = r'C:/Users/mastr/miniconda3/pkgs/proj4-5.2.0-ha925a31_1/Library/share'     ## Windows OS
# os.environ['PROJ_LIB'] = r'/Users/mmastro/miniconda3/pkgs/proj4-5.2.0-ha925a31_1/Library/share'     ## Mac OS
import glob
import netCDF4 as nc
import numpy as np
import pandas as pd
import xarray as xr
#from scipy.signal import argrelextrema                      # Find local Maxima-Minima in numpy array
import matplotlib.colors as colors
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import cartopy.feature as cfeature
import cartopy as cart
import cartopy.crs as ccrs
import cartopy.mpl.ticker as cticker

Functions declaration

In [2]:
def detrend_dim(da, dim, degree):
    # Store original attributes
    original_attrs = da.attrs

    # Detrend along a single dimension
    p = da.polyfit(dim=dim, deg=degree)
    fit = xr.polyval(da[dim], p.polyfit_coefficients)
    da_det = da - fit
    
    # Restore original attributes
    da_det.attrs = original_attrs
    
    return da_det

def xr_mean_list(xarray_list):
    # Step 1: Group the xarray objects by their "esm" attribute
    esm_groups = {}
    for xr_obj in xarray_list:
        esm_value = xr_obj.attrs.get('esm', None)
        if esm_value not in esm_groups:
            esm_groups[esm_value] = []
        esm_groups[esm_value].append(xr_obj)

    # Step 2: Sort the esm groups alphabetically by their esm value
    sorted_esm_values = sorted(esm_groups.keys())

    # Step 3: Calculate the mean for each group in alphabetical order
    mean_results = {}
    for esm_value in sorted_esm_values:
        xr_objs = esm_groups[esm_value]
        
        # Concatenate all xarray objects in this group along a new dimension (e.g., 'stacked_xarrays')
        combined = xr.concat(xr_objs, dim='stacked_xarrays')
        
        # Calculate the mean along the 'stacked_xarrays' dimension
        mean_results[esm_value] = combined.mean(dim='stacked_xarrays')
    
    return mean_results

## Fuction for subsetting colormap values ## 
def truncate_colormap(cmap, minval=0.0, maxval=1.0, n=100):
    new_cmap = colors.LinearSegmentedColormap.from_list(
        'trunc({n},{a:.2f},{b:.2f})'.format(n=cmap.name, a=minval, b=maxval),
        cmap(np.linspace(minval, maxval, n)))
    return new_cmap

## Function for creating a path, if needed ##
def checkDir(out_path):
    if not os.path.exists(out_path):
        os.makedirs(out_path)

def lag_linregress_3D(x, y, lagx=0, lagy=0):
    """
    Input: Two xr.Datarrays of any dimensions with the first dim being time. 
    Thus the input data could be a 1D time series, or for example, have three dimensions (time,lat,lon). 
    Datasets can be provied in any order, but note that the regression slope and intercept will be calculated
    for y with respect to x.
    Output: Covariance, correlation, regression slope and intercept, p-value, and standard error on regression
    between the two datasets along their aligned time dimension.  
    Lag values can be assigned to either of the data, with lagx shifting x, and lagy shifting y, with the specified lag amount. 
    """ 
    #1. Ensure that the data are properly alinged to each other. 
    x,y = xr.align(x,y)
    
    #2. Add lag information if any, and shift the data accordingly
    if lagx!=0:
        #If x lags y by 1, x must be shifted 1 step backwards. 
        #But as the 'zero-th' value is nonexistant, xr assigns it as invalid (nan). Hence it needs to be dropped
        x   = x.shift(time = -lagx).dropna(dim='time')
        #Next important step is to re-align the two datasets so that y adjusts to the changed coordinates of x
        x,y = xr.align(x,y)

    if lagy!=0:
        y   = y.shift(time = -lagy).dropna(dim='time')
        x,y = xr.align(x,y)
 
    #3. Compute data length, mean and standard deviation along time axis for further use: 
    n     = x.shape[0]
    xmean = x.mean(axis=0)
    ymean = y.mean(axis=0)
    xstd  = x.std(axis=0)
    ystd  = y.std(axis=0)
    
    #4. Compute covariance along time axis
    cov   =  np.sum((x - xmean)*(y - ymean), axis=0)/(n)
    
    #5. Compute correlation along time axis
    cor   = cov/(xstd*ystd)
    
    #6. Compute regression slope and intercept:
    slope     = cov/(xstd**2)
    intercept = ymean - xmean*slope
    y_pred =  intercept + slope*x
    res = y - y_pred

    #7. Compute P-value and standard error
    #Compute t-statistics
    tstats = cor*np.sqrt(n-2)/np.sqrt(1-cor**2)
    stderr = slope/tstats
    
    from scipy.stats import t
    pval   = t.sf(tstats, n-2)*2
    pval   = xr.DataArray(pval, dims=cor.dims, coords=cor.coords)

    #return cov,cor,slope,intercept,pval,stderr
    return res

def cor_3D(x, y, lagx=0, lagy=0):
    """
    Input: Two xr.Datarrays of any dimensions with the first dim being time. 
    Thus the input data could be a 1D time series, or for example, have three dimensions (time,lat,lon). 
    Datasets can be provied in any order, but note that the regression slope and intercept will be calculated
    for y with respect to x.
    Output: Covariance, correlation, regression slope and intercept, p-value, and standard error on regression
    between the two datasets along their aligned time dimension.  
    Lag values can be assigned to either of the data, with lagx shifting x, and lagy shifting y, with the specified lag amount. 
    """ 
    #1. Ensure that the data are properly alinged to each other. 
    x,y = xr.align(x,y)
    
    #2. Add lag information if any, and shift the data accordingly
    if lagx!=0:
        #If x lags y by 1, x must be shifted 1 step backwards. 
        #But as the 'zero-th' value is nonexistant, xr assigns it as invalid (nan). Hence it needs to be dropped
        x   = x.shift(time = -lagx).dropna(dim='time')
        #Next important step is to re-align the two datasets so that y adjusts to the changed coordinates of x
        x,y = xr.align(x,y)

    if lagy!=0:
        y   = y.shift(time = -lagy).dropna(dim='time')
        x,y = xr.align(x,y)
 
    #3. Compute data length, mean and standard deviation along time axis for further use: 
    n     = x.shape[0]
    xmean = x.mean(axis=0)
    ymean = y.mean(axis=0)
    xstd  = x.std(axis=0)
    ystd  = y.std(axis=0)
    
    #4. Compute covariance along time axis
    cov   =  np.sum((x - xmean)*(y - ymean), axis=0)/(n)
    
    #5. Compute correlation along time axis
    cor   = cov/(xstd*ystd)

    #return cov,cor,slope,intercept,pval,stderr
    return cor

def xr_multipletest(p, alpha=0.05, method='fdr_bh', **multipletests_kwargs):
    """Apply statsmodels.stats.multitest.multipletests for multi-dimensional xr.objects."""
    from statsmodels.stats.multitest import multipletests
    # stack all to 1d array
    p_stacked = p.stack(s=p.dims)
    # mask only where not nan: https://github.com/statsmodels/statsmodels/issues/2899
    mask = np.isfinite(p_stacked)
    pvals_corrected = np.full(p_stacked.shape, np.nan)
    reject = np.full(p_stacked.shape, np.nan)
    # apply test where mask
    reject[mask] = multipletests(
        p_stacked[mask], alpha=alpha, method=method, **multipletests_kwargs)[0]
    pvals_corrected[mask] = multipletests(
        p_stacked[mask], alpha=alpha, method=method, **multipletests_kwargs)[1]

    def unstack(reject, p_stacked):
        """Exchange values from p_stacked with reject (1d array) and unstack."""
        xreject = p_stacked.copy()
        xreject.values = reject
        xreject = xreject.unstack()
        return xreject

    reject = unstack(reject, p_stacked)
    pvals_corrected = unstack(pvals_corrected, p_stacked)
    return reject, pvals_corrected


In [3]:
# -- Path for the output files (images, etc)
# out_path = 'C:/Users/mastr/Documents/Amazon/RESULTS/'
out_path = 'G:/Shared drives/Amazon_ENSO_work/RESULTS/MLR/'

#out_path = "D:/Data/CMIP6/RESULTS"

# -- Create directories
checkDir(out_path)

### Open SST data

In [4]:
##### ------- Open data (MODEL) ------- #####
data_path = 'F:/Data/analysis/'
scenario = 'historical'
files = "nino34" + '_*_' + scenario + '_*'

files_list = []
for filepath in sorted(glob.glob(os.path.join(data_path+'/'+scenario+'/'+files))):               # List of files sorted by name
        content = nc.Dataset(filepath)
        files_list.append(content)   

nino34_hist = []
for filepath in sorted(glob.glob(os.path.join(data_path+'/'+scenario+'/'+files))):               # sorted is case sensitive                             ## List of files sorted by name
    content = xr.open_dataset(filepath, drop_variables=["time_bnds","lon_bnds","lat_bnds"]); #content = content.mean(dim="time")         ## values   var     dims    coords
    nino34_hist.append(content)

scenario = 'ssp585'
files = "nino34" + '_*_' + scenario + '_*' 

nino34_ssp = []
for filepath in sorted(glob.glob(os.path.join(data_path+'/'+scenario+'/'+files))):               # sorted is case sensitive                             ## List of files sorted by name
    content = xr.open_dataset(filepath, drop_variables=["time_bnds","lon_bnds","lat_bnds"]); #content = content.mean(dim="time")         ## values   var     dims    coords
    nino34_ssp.append(content)

## Normalization
# for i,item in enumerate(nino34_hist):
#     nino34_hist[i] =  ((nino34_hist[i] - (nino34_hist[i].mean(dim='time'))).compute()/(nino34_hist[i].std(dim='time'))).compute()

# for i,item in enumerate(nino34_ssp):
#     nino34_ssp[i] =  ((nino34_ssp[i] - (nino34_ssp[i].mean(dim='time'))).compute()/(nino34_ssp[i].std(dim='time'))).compute()


## Resample from Monthly to seasonal timesteps
nino34_hist = [a.resample(time="Y", label='left').mean() for a in nino34_hist]
nino34_ssp = [a.resample(time="Y", label='left').mean() for a in nino34_ssp]

# Convert to dataarray
nino34_hist = [a.to_array() for a in nino34_hist]
nino34_ssp = [a.to_array() for a in nino34_ssp]

# Delete useless empty dimension
nino34_hist = [nino.squeeze().rename(variable = "tos").drop("tos") for nino in nino34_hist]
nino34_ssp = [nino.squeeze().rename(variable = "tos").drop("tos") for nino in nino34_ssp]

# Correct for spurious dimension
for i, item in enumerate(nino34_hist):
    if len(nino34_hist[i].shape)!=1:
        nino34_hist[i] = nino34_hist[i][1]
    else:
        None


### Open LAND data

In [5]:
data_path = 'F:/Data/analysis/'              

scenario = 'historical'
var_name = 'nbp'
files = var_name + '_*_' + scenario + '_*' 

files_list = []
for filepath in sorted(glob.glob(os.path.join(data_path+'/'+scenario+'/'+files))):               # List of files sorted by name
        content = nc.Dataset(filepath)
        files_list.append(content)                                              # to retrieve netcdf original ATTRIBUTES

ds_hist_nbp = []
for filepath in sorted(glob.glob(os.path.join(data_path+'/'+scenario+'/'+files))):                                       ## List of files sorted by name
    content = xr.open_dataarray(filepath, drop_variables=["time_bnds","lon_bnds","lat_bnds"])          ## values   var     dims    coords
    content = content.assign_attrs(esm=nc.Dataset(filepath).source_id)
    ds_hist_nbp.append(content)

scenario = 'ssp585'
files = var_name + '_*_' + scenario + '_*' 

ds_ssp_nbp = []
for filepath in sorted(glob.glob(os.path.join(data_path+'/'+scenario+'/'+files))):                                       ## List of files sorted by name
    content = xr.open_dataarray(filepath, drop_variables=["time_bnds","lon_bnds","lat_bnds"])           ## values   var     dims    coords
    content = content.assign_attrs(esm=nc.Dataset(filepath).source_id)
    ds_ssp_nbp.append(content)  

# Subsetting latitude
ds_hist_nbp = [a.sel(lat=slice(-30,30)) for a in ds_hist_nbp]
ds_ssp_nbp = [a.sel(lat=slice(-30,30)) for a in ds_ssp_nbp]

# Uniform calendar
for i, item in enumerate(ds_hist_nbp):
    if item['time'].dt.calendar == 'noleap' or item['time'].dt.calendar == '360_day':
        ds_hist_nbp[i]['time'] = item.indexes['time'].to_datetimeindex()

for i, item in enumerate(ds_ssp_nbp):
    if item['time'].dt.calendar == 'noleap' or item['time'].dt.calendar == '360_day':
        ds_ssp_nbp[i]['time'] = item.indexes['time'].to_datetimeindex()

# Select time periods
ds_hist_nbp = [a for a in ds_hist_nbp]
ds_ssp_nbp = [a for a in ds_ssp_nbp]

# Detrending of 1st order
ds_hist_nbp = [detrend_dim(a, "time", 1) for a in ds_hist_nbp]
ds_ssp_nbp = [detrend_dim(a, "time", 1) for a in ds_ssp_nbp]

# Resample from months to seasons
ds_hist_nbp = [a.resample(time="Y", label='left').mean() for a in ds_hist_nbp]
ds_ssp_nbp = [a.resample(time="Y", label='left').mean() for a in ds_ssp_nbp]

# CanESM5 hist and ssp have numerical values of ocean equal to 
# hist: -9.304011e-07
# ssp: -1.120609e-05

for i, item in enumerate(files_list):
    if files_list[i].source_id == "CanESM5":
        ds_hist_nbp[i] = ds_hist_nbp[i].where(ds_hist_nbp[i] != -3.7270379e-07)
        ds_ssp_nbp[i] = ds_ssp_nbp[i].where(ds_ssp_nbp[i] != -6.18386321e-06)

In [6]:
data_path = 'F:/Data/analysis/'              

scenario = 'historical'
var_name = 'pr'
files = var_name + '_*_' + scenario + '_*' 

ds_hist_pr = []
for filepath in sorted(glob.glob(os.path.join(data_path+'/'+scenario+'/'+files))):                                       ## List of files sorted by name
    content = xr.open_dataarray(filepath, drop_variables=["time_bnds","lon_bnds","lat_bnds"])          ## values   var     dims    coords
    content = content.assign_attrs(esm=nc.Dataset(filepath).source_id)
    ds_hist_pr.append(content)

scenario = 'ssp585'
files = var_name + '_*_' + scenario + '_*' 

ds_ssp_pr = []
for filepath in sorted(glob.glob(os.path.join(data_path+'/'+scenario+'/'+files))):                                       ## List of files sorted by name
    content = xr.open_dataarray(filepath, drop_variables=["time_bnds","lon_bnds","lat_bnds"])           ## values   var     dims    coords
    content = content.assign_attrs(esm=nc.Dataset(filepath).source_id)
    ds_ssp_pr.append(content)   

# Uniform calendar
for i, item in enumerate(ds_hist_pr):
    if item['time'].dt.calendar == 'noleap' or item['time'].dt.calendar == '360_day':
        ds_hist_pr[i]['time'] = item.indexes['time'].to_datetimeindex()

for i, item in enumerate(ds_ssp_pr):
    if item['time'].dt.calendar == 'noleap' or item['time'].dt.calendar == '360_day':
        ds_ssp_pr[i]['time'] = item.indexes['time'].to_datetimeindex()

# Subsetting latitude
ds_hist_pr = [a.sel(lat=slice(-30,30)) for a in ds_hist_pr]
ds_ssp_pr = [a.sel(lat=slice(-30,30)) for a in ds_ssp_pr]

# Correct pr ssp585 UKESM-r4 from lon:192 to lon:191
ds_ssp_pr[39] = ds_ssp_pr[39].isel(lon = slice(0,191))
# Correct pr hist UKESM-r4 from lon:192 to lon:191
ds_hist_pr[39] = ds_hist_pr[39].isel(lon = slice(0,191))

# Select time periods
ds_hist_pr = [a for a in ds_hist_pr]
ds_ssp_pr = [a for a in ds_ssp_pr]

# Detrending of 1st order
ds_hist_pr = [detrend_dim(a, "time", 1) for a in ds_hist_pr]
ds_ssp_pr = [detrend_dim(a, "time", 1) for a in ds_ssp_pr]

# Resample from months to seasons
ds_hist_pr = [a.resample(time="Y", label='left').sum() for a in ds_hist_pr]
ds_ssp_pr = [a.resample(time="Y", label='left').sum() for a in ds_ssp_pr]

# CanESM5 hist and ssp have numerical values of ocean equal to 
# hist: -9.304011e-07
# ssp: -1.120609e-05

for i, item in enumerate(files_list):
    if files_list[i].source_id == "CanESM5":
        ds_hist_pr[i] = ds_hist_pr[i].where(ds_hist_pr[i] != -3.7270379e-07)
        ds_ssp_pr[i] = ds_ssp_pr[i].where(ds_ssp_pr[i] != -6.18386321e-06)

In [7]:
data_path = 'F:/Data/analysis/'              

scenario = 'historical'
var_name = 'tas'
files = var_name + '_*_' + scenario + '_*' 

ds_hist_tas = []
for filepath in sorted(glob.glob(os.path.join(data_path+'/'+scenario+'/'+files))):                                       ## List of files sorted by name
    content = xr.open_dataarray(filepath, drop_variables=["time_bnds","lon_bnds","lat_bnds"])          ## values   var     dims    coords
    content = content.assign_attrs(esm=nc.Dataset(filepath).source_id)
    ds_hist_tas.append(content)

scenario = 'ssp585'
files = var_name + '_*_' + scenario + '_*' 

ds_ssp_tas = []
for filepath in sorted(glob.glob(os.path.join(data_path+'/'+scenario+'/'+files))):                                       ## List of files sorted by name
    content = xr.open_dataarray(filepath, drop_variables=["time_bnds","lon_bnds","lat_bnds"])           ## values   var     dims    coords
    content = content.assign_attrs(esm=nc.Dataset(filepath).source_id)
    ds_ssp_tas.append(content)  

# Uniform calendar
for i, item in enumerate(ds_hist_tas):
    if item['time'].dt.calendar == 'noleap' or item['time'].dt.calendar == '360_day':
        ds_hist_tas[i]['time'] = item.indexes['time'].to_datetimeindex()

for i, item in enumerate(ds_ssp_tas):
    if item['time'].dt.calendar == 'noleap' or item['time'].dt.calendar == '360_day':
        ds_ssp_tas[i]['time'] = item.indexes['time'].to_datetimeindex()

# Subsetting latitude
ds_hist_tas = [a.sel(lat=slice(-30,30)) for a in ds_hist_tas]
ds_ssp_tas = [a.sel(lat=slice(-30,30)) for a in ds_ssp_tas]

# Select time periods
ds_hist_tas = [a for a in ds_hist_tas]
ds_ssp_tas = [a for a in ds_ssp_tas]

# Detrending of 1st order
ds_hist_tas = [detrend_dim(a, "time", 1) for a in ds_hist_tas]
ds_ssp_tas = [detrend_dim(a, "time", 1) for a in ds_ssp_tas]

# Resample from months to seasons
ds_hist_tas = [a.resample(time="Y", label='left').mean() for a in ds_hist_tas]
ds_ssp_tas = [a.resample(time="Y", label='left').mean() for a in ds_ssp_tas]

# CanESM5 hist and ssp have numerical values of ocean equal to 
# hist: -9.304011e-07
# ssp: -1.120609e-05

for i, item in enumerate(files_list):
    if files_list[i].source_id == "CanESM5":
        ds_hist_tas[i] = ds_hist_tas[i].where(ds_hist_tas[i] != -3.7270379e-07)
        ds_ssp_tas[i] = ds_ssp_tas[i].where(ds_ssp_tas[i] != -6.18386321e-06)

In [8]:
data_path = 'F:/Data/analysis/'              

scenario = 'historical'
var_name = 'mrso'
files = var_name + '_*_' + scenario + '_*' 

ds_hist_mrso = []
for filepath in sorted(glob.glob(os.path.join(data_path+'/'+scenario+'/'+files))):                                       ## List of files sorted by name
    content = xr.open_dataarray(filepath, drop_variables=["time_bnds","lon_bnds","lat_bnds"])          ## values   var     dims    coords
    content = content.assign_attrs(esm=nc.Dataset(filepath).source_id)
    ds_hist_mrso.append(content)

scenario = 'ssp585'
files = var_name + '_*_' + scenario + '_*' 

ds_ssp_mrso = []
for filepath in sorted(glob.glob(os.path.join(data_path+'/'+scenario+'/'+files))):                                       ## List of files sorted by name
    content = xr.open_dataarray(filepath, drop_variables=["time_bnds","lon_bnds","lat_bnds"])           ## values   var     dims    coords
    content = content.assign_attrs(esm=nc.Dataset(filepath).source_id)
    ds_ssp_mrso.append(content)   

# Subsetting latitude
ds_hist_mrso = [a.sel(lat=slice(-30,30)) for a in ds_hist_mrso]
ds_ssp_mrso = [a.sel(lat=slice(-30,30)) for a in ds_ssp_mrso]

# Uniform calendar
for i, item in enumerate(ds_hist_mrso):
    if item['time'].dt.calendar == 'noleap' or item['time'].dt.calendar == '360_day':
        ds_hist_mrso[i]['time'] = item.indexes['time'].to_datetimeindex()

for i, item in enumerate(ds_ssp_mrso):
    if item['time'].dt.calendar == 'noleap' or item['time'].dt.calendar == '360_day':
        ds_ssp_mrso[i]['time'] = item.indexes['time'].to_datetimeindex()

# Select time periods
ds_hist_mrso = [a for a in ds_hist_mrso]
ds_ssp_mrso = [a for a in ds_ssp_mrso]

# Detrending of 1st order
ds_hist_mrso = [detrend_dim(a, "time", 1) for a in ds_hist_mrso]
ds_ssp_mrso = [detrend_dim(a, "time", 1) for a in ds_ssp_mrso]

# Resample from months to seasons
ds_hist_mrso = [a.resample(time="Y", label='left').mean() for a in ds_hist_mrso]
ds_ssp_mrso = [a.resample(time="Y", label='left').mean() for a in ds_ssp_mrso]


In [9]:
data_path = 'F:/Data/analysis/'              

scenario = 'historical'
var_name = 'rsds'
files = var_name + '_*_' + scenario + '_*' 

ds_hist_rsds = []
for filepath in sorted(glob.glob(os.path.join(data_path+'/'+scenario+'/'+files))):                                       ## List of files sorted by name
    content = xr.open_dataarray(filepath, drop_variables=["time_bnds","lon_bnds","lat_bnds"])          ## values   var     dims    coords
    content = content.assign_attrs(esm=nc.Dataset(filepath).source_id)
    ds_hist_rsds.append(content)

scenario = 'ssp585'
files = var_name + '_*_' + scenario + '_*' 

ds_ssp_rsds = []
for filepath in sorted(glob.glob(os.path.join(data_path+'/'+scenario+'/'+files))):                                       ## List of files sorted by name
    content = xr.open_dataarray(filepath, drop_variables=["time_bnds","lon_bnds","lat_bnds"])           ## values   var     dims    coords
    content = content.assign_attrs(esm=nc.Dataset(filepath).source_id)
    ds_ssp_rsds.append(content)    

# Subsetting latitude
ds_hist_rsds = [a.sel(lat=slice(-30,30)) for a in ds_hist_rsds]
ds_ssp_rsds = [a.sel(lat=slice(-30,30)) for a in ds_ssp_rsds]

# Uniform calendar
for i, item in enumerate(ds_hist_rsds):
    if item['time'].dt.calendar == 'noleap' or item['time'].dt.calendar == '360_day':
        ds_hist_rsds[i]['time'] = item.indexes['time'].to_datetimeindex()

for i, item in enumerate(ds_ssp_rsds):
    if item['time'].dt.calendar == 'noleap' or item['time'].dt.calendar == '360_day':
        ds_ssp_rsds[i]['time'] = item.indexes['time'].to_datetimeindex()

# Select time periods
ds_hist_rsds = [a for a in ds_hist_rsds]
ds_ssp_rsds = [a for a in ds_ssp_rsds]

# Detrending of 1st order
ds_hist_rsds = [detrend_dim(a, "time", 1) for a in ds_hist_rsds]
ds_ssp_rsds = [detrend_dim(a, "time", 1) for a in ds_ssp_rsds]

# Resample from months to seasons
ds_hist_rsds = [a.resample(time="Y", label='left').mean() for a in ds_hist_rsds]
ds_ssp_rsds = [a.resample(time="Y", label='left').mean() for a in ds_ssp_rsds]


In [10]:
data_path = 'F:/Data/analysis/'              

scenario = 'historical'
var_name = 'hurs'
files = var_name + '_*_' + scenario + '_*' 

ds_hist_hurs = []
for filepath in sorted(glob.glob(os.path.join(data_path+'/'+scenario+'/'+files))):                                       ## List of files sorted by name
    content = xr.open_dataarray(filepath, drop_variables=["time_bnds","lon_bnds","lat_bnds"])          ## values   var     dims    coords
    content = content.assign_attrs(esm=nc.Dataset(filepath).source_id)
    ds_hist_hurs.append(content)

scenario = 'ssp585'
files = var_name + '_*_' + scenario + '_*' 

ds_ssp_hurs = []
for filepath in sorted(glob.glob(os.path.join(data_path+'/'+scenario+'/'+files))):                                       ## List of files sorted by name
    content = xr.open_dataarray(filepath, drop_variables=["time_bnds","lon_bnds","lat_bnds"])           ## values   var     dims    coords
    content = content.assign_attrs(esm=nc.Dataset(filepath).source_id)    
    ds_ssp_hurs.append(content)    

# Subsetting latitude
ds_hist_hurs = [a.sel(lat=slice(-30,30)) for a in ds_hist_hurs]
ds_ssp_hurs = [a.sel(lat=slice(-30,30)) for a in ds_ssp_hurs]

# Uniform calendar
for i, item in enumerate(ds_hist_hurs):
    if item['time'].dt.calendar == 'noleap' or item['time'].dt.calendar == '360_day':
        ds_hist_hurs[i]['time'] = item.indexes['time'].to_datetimeindex()

for i, item in enumerate(ds_ssp_hurs):
    if item['time'].dt.calendar == 'noleap' or item['time'].dt.calendar == '360_day':
        ds_ssp_hurs[i]['time'] = item.indexes['time'].to_datetimeindex()

# Select time periods
ds_hist_hurs = [a for a in ds_hist_hurs]
ds_ssp_hurs = [a for a in ds_ssp_hurs]

# Detrending of 1st order
ds_hist_hurs = [detrend_dim(a, "time", 1) for a in ds_hist_hurs]
ds_ssp_hurs = [detrend_dim(a, "time", 1) for a in ds_ssp_hurs]

# Resample from months to seasons
ds_hist_hurs = [a.resample(time="Y", label='left').mean() for a in ds_hist_hurs]
ds_ssp_hurs = [a.resample(time="Y", label='left').mean() for a in ds_ssp_hurs]


In [16]:
import math

def calculate_vpd(rh, tas, unit):
    """
    Input: Relative Humidity and Air Temperature 
    Output: Vapour Pressure Deficit (VPD), in kPa
    
    Parameters:
    rh : Relative Humidity in %
    tas : Air temperature in Kelvin or Celsius
    unit : 'Kelvin' or 'Celsius'
    """
    original_attrs = tas.attrs
    # Convert temperature to Celsius if given in Kelvin
    if unit == "Kelvin":
        tas_celsius = tas - 273.15
    elif unit == "Celsius":
        tas_celsius = tas
    else:
        raise ValueError("Insert correct Unit of Measure for Temperature: 'Kelvin' or 'Celsius'")
    
    # Calculate Saturation Vapour Pressure (SVP) in kPa
    svp = 0.61078 * np.exp((17.27 * tas_celsius) / (tas_celsius + 237.3))
    # Calculate Vapour Pressure Deficit (VPD) in kPa
    vpd = svp * (1 - rh / 100)
    vpd = vpd.assign_attrs(original_attrs)
    vpd = vpd.assign_attrs(unit = "kPa")

    return vpd

In [17]:
ds_hist_vpd = []; ds_ssp_vpd = []
for i, item in enumerate(ds_hist_hurs):
    content = calculate_vpd(ds_hist_hurs[i], ds_hist_tas[i], "Celsius")
    content1 = calculate_vpd(ds_ssp_hurs[i], ds_ssp_tas[i], "Celsius")

    ds_hist_vpd.append(content)
    ds_ssp_vpd.append(content)

In [18]:
for i, item in enumerate(ds_hist_vpd):
    if item.esm == "NorESM2-LM":
        ds_hist_vpd[i] = ds_hist_vpd[i].isel(plev = 0)
        ds_ssp_vpd[i] = ds_ssp_vpd[i].isel(plev = 0)

In [19]:
del ds_hist_hurs
del ds_ssp_hurs

### Correct for lon lat mishape and values

In [20]:
# Correct the number of longitude points
ds_hist_nbp = [a.isel(lon = slice(0,b.lon.shape[0])) for a,b in zip(ds_hist_nbp, ds_hist_vpd)]
ds_hist_mrso = [a.isel(lon = slice(0,b.lon.shape[0])) for a,b in zip(ds_hist_mrso, ds_hist_vpd)]
ds_hist_tas = [a.isel(lon = slice(0,b.lon.shape[0])) for a,b in zip(ds_hist_tas, ds_hist_vpd)]
ds_hist_rsds = [a.isel(lon = slice(0,b.lon.shape[0])) for a,b in zip(ds_hist_rsds, ds_hist_vpd)]
ds_hist_pr = [a.isel(lon = slice(0,b.lon.shape[0])) for a,b in zip(ds_hist_pr, ds_hist_vpd)]
ds_ssp_vpd = [a.isel(lon = slice(0,b.lon.shape[0])) for a,b in zip(ds_ssp_vpd, ds_ssp_pr)]
ds_ssp_rsds = [a.isel(lon = slice(0,b.lon.shape[0])) for a,b in zip(ds_ssp_rsds, ds_ssp_pr)]
ds_ssp_nbp = [a.isel(lon = slice(0,b.lon.shape[0])) for a,b in zip(ds_ssp_nbp, ds_ssp_pr)]
ds_ssp_mrso = [a.isel(lon = slice(0,b.lon.shape[0])) for a,b in zip(ds_ssp_mrso, ds_ssp_pr)]
ds_ssp_tas = [a.isel(lon = slice(0,b.lon.shape[0])) for a,b in zip(ds_ssp_tas, ds_ssp_pr)]

# Assign the exactly same set of coordinates to ALL the dataarray (avoid duplication due to numerical approximation of lon lat)
ds_hist_pr = [a.assign_coords(lat = b.lat, lon = b.lon) for a,b in zip(ds_hist_pr,ds_hist_nbp)]
ds_hist_tas = [a.assign_coords(lat = b.lat, lon = b.lon) for a,b in zip(ds_hist_tas,ds_hist_nbp)]
ds_hist_mrso = [a.assign_coords(lat = b.lat, lon = b.lon) for a,b in zip(ds_hist_mrso,ds_hist_nbp)]
ds_hist_rsds = [a.assign_coords(lat = b.lat, lon = b.lon) for a,b in zip(ds_hist_rsds,ds_hist_nbp)]
ds_hist_vpd = [a.assign_coords(lat = b.lat, lon = b.lon) for a,b in zip(ds_hist_vpd,ds_hist_nbp)]
ds_ssp_vpd = [a.assign_coords(lat = b.lat, lon = b.lon) for a,b in zip(ds_ssp_vpd,ds_ssp_nbp)]
ds_ssp_rsds = [a.assign_coords(lat = b.lat, lon = b.lon) for a,b in zip(ds_ssp_rsds,ds_ssp_nbp)]
ds_ssp_pr = [a.assign_coords(lat = b.lat, lon = b.lon) for a,b in zip(ds_ssp_pr,ds_ssp_nbp)]
ds_ssp_tas = [a.assign_coords(lat = b.lat, lon = b.lon) for a,b in zip(ds_ssp_tas,ds_ssp_nbp)]
ds_ssp_mrso = [a.assign_coords(lat = b.lat, lon = b.lon) for a,b in zip(ds_ssp_mrso,ds_ssp_nbp)]

### Ridge Reg with ENSO signal

ENSO is linearly removed from original variables signal, The residuals are then used in the ridge regression 

In [21]:
# Estimate the residuals of regression with nino34 to isolate its effect on pr and tas
seas = "DJF"

# Reshape variables
X1 = [a for a in nino34_hist]
Y = [a for a in ds_hist_pr]
Y1 = [a for a in ds_hist_tas]
Y2 = [a for a in ds_hist_mrso]
Y3 = [a for a in ds_hist_rsds]
Y4 = [a for a in ds_hist_vpd]

res_pr_hist = []
res_tas_hist = []
res_mrso_hist = []
res_rsds_hist = []
res_vpd_hist = []
for i, item in enumerate(ds_hist_nbp):

    content = lag_linregress_3D(X1[i],Y[i])
    content1 = lag_linregress_3D(X1[i],Y1[i])
    content2 = lag_linregress_3D(X1[i],Y2[i])
    content3 = lag_linregress_3D(X1[i],Y3[i])
    content4 = lag_linregress_3D(X1[i],Y4[i])

    res_pr_hist.append(content)
    res_tas_hist.append(content1)
    res_mrso_hist.append(content2)
    res_rsds_hist.append(content3)
    res_vpd_hist.append(content4)

# Reshape variables
X1 = [a for a in nino34_ssp]
Y = [a for a in ds_ssp_pr]
Y1 = [a for a in ds_ssp_tas]
Y2 = [a for a in ds_ssp_mrso]
Y3 = [a for a in ds_ssp_rsds]
Y4 = [a for a in ds_ssp_vpd]

res_pr_ssp = []
res_tas_ssp = []
res_mrso_ssp = []
res_rsds_ssp = []
res_vpd_ssp = []
for i, item in enumerate(ds_ssp_nbp):

    content = lag_linregress_3D(X1[i],Y[i])
    content1 = lag_linregress_3D(X1[i],Y1[i])
    content2 = lag_linregress_3D(X1[i],Y2[i])
    content3 = lag_linregress_3D(X1[i],Y3[i])
    content4 = lag_linregress_3D(X1[i],Y4[i])

    res_pr_ssp.append(content)
    res_tas_ssp.append(content1)
    res_mrso_ssp.append(content2)
    res_rsds_ssp.append(content3)
    res_vpd_ssp.append(content4)

### Calculation

To avoid Singular Matrix error while performing regression it's necessary to fill NaN NOT with zero but with a not-zero number (eg 0.001) 

By standardizing Soil Moisturre SSP with historical Soil Moisture we get infinite values for some realization. Thus substitute inf with 0.001

In [ ]:
from sklearn import linear_model
from regressors import stats   

# Define regression technique
# Ridge Regression with Cross-Validation
alphas = np.linspace(.000001, 1)
alphas = np.logspace(-6, 2, 10)
ridge = linear_model.RidgeCV(alphas = alphas, cv =5)

# Ridge Regression without Cross-Validation
# ridge = linear_model.Ridge()

# historical
coef_ridge_hist = []
r2_ridge_hist = []
pred_ridge_hist = []
pval_ridge_hist = []
for i, item in enumerate(ds_hist_nbp):
        print("processing historical_" + ds_hist_nbp[i].esm)

    # if esm[i] == 'CESM2-WACCM':
        
        #1. Reshape variables
        ensohist = nino34_hist[i].rename("sst")
        X1hist = res_pr_hist[i].rename("pr").sel(lon= slice(260,340))
        X2hist = res_tas_hist[i].rename("tas").sel(lon= slice(260,340))
        X3hist = res_mrso_hist[i].rename("mrso").fillna(0.001).sel(lon= slice(260,340))
        X4hist = res_rsds_hist[i].rename("rsds").sel(lon= slice(260,340))
        X5hist = res_vpd_hist[i].rename("vpd").sel(lon= slice(260,340))

        Y = ds_hist_nbp[i].sel(lon= slice(260,340)).fillna(0.001)

        #2. Standardize predictors
        ensohist = ensohist/ensohist.std(dim = ["time"])
        X1hist = X1hist/X1hist.std(dim = "time")
        X2hist = X2hist/X2hist.std(dim = "time")
        X3hist = (X3hist/X3hist.std(dim = "time")).fillna(0.001)
        X4hist = (X4hist/X4hist.std(dim = "time"))
        X5hist = X5hist/X5hist.std(dim = "time")

        #3. Stack on 1D vector
        ensohist = ensohist.stack(cell = ["time"])
        X1hist = X1hist.stack(cell = ["lon","lat"])
        X2hist = X2hist.stack(cell = ["lon","lat"])
        X3hist = X3hist.stack(cell = ["lon","lat"])
        X4hist = X4hist.stack(cell = ["lon","lat"])
        X5hist = X5hist.stack(cell = ["lon","lat"])

        Y = Y.stack(cell = ["lon","lat"])

        #4. Create Dataarray with ENSO signal in every grid cell to perform regression
        Xenso = xr.DataArray(data=None, coords=[X1hist.time, X1hist.cell], dims=["time","cell"])
        for c in Xenso.cell:
            locator = {'cell':c}
            Xenso.loc[locator] = ensohist.values
            Xenso.values = Xenso.values.astype("float64")       # To avoid the resulting values are "object" rather than "float64" 
        Xenso = Xenso.rename("sst")

        #5. Create empty dataarray to store regression coefficients
        coef = xr.DataArray(data=None, coords=[X1hist.time,X1hist.cell], dims=["time","cell"])
        # coef["pr"] =  xr.DataArray(data=None, coords=[X1hist.cell], dims=["cell"])
        coef["tas"] =  xr.DataArray(data=None, coords=[X1hist.cell], dims=["cell"])
        coef["mrso"] =  xr.DataArray(data=None, coords=[X1hist.cell], dims=["cell"])
        coef["sst"] =  xr.DataArray(data=None, coords=[X1hist.cell], dims=["cell"])
        coef["rsds"] =  xr.DataArray(data=None, coords=[X1hist.cell], dims=["cell"])
        coef["vpd"] =  xr.DataArray(data=None, coords=[X1hist.cell], dims=["cell"])

        # Create empty dataarray to store p-values
        pval = xr.DataArray(data=None, coords=[X1hist.cell], dims=["cell"])
        # pval["pr"] = xr.DataArray(data=None, coords=[X1hist.cell], dims=["cell"])
        pval["tas"] = xr.DataArray(data=None, coords=[X1hist.cell], dims=["cell"])
        pval["mrso"] = xr.DataArray(data=None, coords=[X1hist.cell], dims=["cell"])
        pval["sst"] = xr.DataArray(data=None, coords=[X1hist.cell], dims=["cell"])
        pval["rsds"] =  xr.DataArray(data=None, coords=[X1hist.cell], dims=["cell"])
        pval["vpd"] =  xr.DataArray(data=None, coords=[X1hist.cell], dims=["cell"])

        # Create empty datarray to store R2 and Y predicted 
        r_squared = xr.DataArray(data=None, coords=[X1hist.cell], dims=["cell"])
        pred = xr.DataArray(data=None, coords=[Y.time, Y.cell], dims=["time","cell"])
        pred["nbp"] = xr.DataArray(data=None, coords=[Y.time, Y.cell], dims=["time", "cell"])

        #6. Iterate over cells (lan*lon)
        for c in X1hist.cell:
            locator = {'cell':c}
            # Check if the mrso value is NaN or 0 for this cell
            if np.isnan(res_mrso_hist[i].stack(cell = ["lon","lat"]).loc[locator].values).any() or (res_mrso_hist[i].stack(cell = ["lon","lat"]).loc[locator].values == 0).any():

                # Assign NaN to all result arrays for this cell
                coef["tas"].loc[locator] = np.nan
                coef["mrso"].loc[locator] = np.nan
                coef["rsds"].loc[locator] = np.nan
                coef["vpd"].loc[locator] = np.nan

                pval["tas"].loc[locator] = np.nan
                pval["mrso"].loc[locator] = np.nan
                pval["rsds"].loc[locator] = np.nan
                pval["vpd"].loc[locator] = np.nan

                r_squared.loc[locator] = np.nan
                pred["nbp"].loc[locator] = np.nan
                continue  # Skip this cell and move to the next

            # merge predictors in one numpy array
            df = np.array((X2hist.loc[locator].values, X3hist.loc[locator].values, X4hist.loc[locator].values, X5hist.loc[locator].values, Xenso.loc[locator].values)); df = df.T
            
            model = ridge.fit(df,Y.loc[locator].values)
            r2 = ridge.score(df,Y.loc[locator])
            ypred = ridge.predict(df)

            # coef["pr"].loc[locator] = model.coef_[0]
            coef["tas"].loc[locator] = model.coef_[0]
            coef["mrso"].loc[locator] = model.coef_[1]
            coef["rsds"].loc[locator] = model.coef_[2]
            coef["vpd"].loc[locator] = model.coef_[3]
            coef["sst"].loc[locator] = model.coef_[4]

            # # p-value of the index "0" refers to the intercept
            # pval["pr"].loc[locator] = stats.coef_pval(model,df,Y.loc[locator].values)[1]
            pval["tas"].loc[locator] = stats.coef_pval(model,df,Y.loc[locator].values)[1]
            pval["mrso"].loc[locator] = stats.coef_pval(model,df,Y.loc[locator].values)[2]
            pval["rsds"].loc[locator] = stats.coef_pval(model,df,Y.loc[locator].values)[3]
            pval["vpd"].loc[locator] = stats.coef_pval(model,df,Y.loc[locator].values)[4]
            pval["sst"].loc[locator] = stats.coef_pval(model,df,Y.loc[locator].values)[5]
            
            r_squared.loc[locator] = r2
            pred["nbp"].loc[locator] = ypred

        #7. Unstack array
        coef = coef.unstack()
        coef = xr.concat([coef.tas, coef.mrso, coef.rsds, coef.vpd, coef.sst], dim = "coefficients")     #ORDER of the coefficients
        # coef["pr"] = coef.pr.astype(np.float64)
        coef["tas"] = coef.tas.astype(np.float64)
        coef["mrso"] = coef.mrso.astype(np.float64)
        coef["rsds"] = coef.rsds.astype(np.float64)
        coef["vpd"] = coef.vpd.astype(np.float64)
        coef["sst"] = coef.sst.astype(np.float64)

        pval = pval.unstack()
        pval = xr.concat([pval.tas, pval.mrso, pval.rsds, pval.vpd, pval.sst], dim = "p-values")
        # pval["pr"] = pval.pr.astype(np.float64)
        pval["tas"] = pval.tas.astype(np.float64)
        pval["mrso"] = pval.mrso.astype(np.float64)
        pval["rsds"] = pval.rsds.astype(np.float64)
        pval["vpd"] = pval.vpd.astype(np.float64)
        pval["sst"] = pval.sst.astype(np.float64)
        
        r_squared = r_squared.astype(np.float64).unstack()
        pred = pred.astype(np.float64).unstack()

        coef_ridge_hist.append(coef)
        r2_ridge_hist.append(r_squared)
        pred_ridge_hist.append(pred)
        pval_ridge_hist.append(pval)

    # else:
    #     None

# SSP585
coef_ridge_ssp = []
r2_ridge_ssp = []
pred_ridge_ssp = []
pval_ridge_ssp = []
for i, item in enumerate(ds_ssp_nbp):
        print("processing ssp585_" + ds_ssp_nbp[i].esm)

    # if esm[i] == 'CESM2-WACCM':

        # Reshape variables
        ensossp = nino34_ssp[i].rename("sst")
        X1ssp = res_pr_ssp[i].rename("pr").sel(lon= slice(260,340))
        X2ssp = res_tas_ssp[i].rename("tas").sel(lon= slice(260,340))
        X3ssp = res_mrso_ssp[i].rename("mrso").fillna(0.001).sel(lon= slice(260,340))
        X4ssp = res_rsds_ssp[i].rename("rsds").sel(lon= slice(260,340))
        X5ssp = res_vpd_ssp[i].rename("vpd").sel(lon= slice(260,340))

        Y = ds_ssp_nbp[i].sel(lon= slice(260,340)).fillna(0.001)

        # Need to redefine historical dataset for standardization 
        ensohist = nino34_hist[i].rename("sst")
        X1hist = res_pr_hist[i].rename("pr").sel(lon= slice(260,340))
        X2hist = res_tas_hist[i].rename("tas").sel(lon= slice(260,340))
        X3hist = res_mrso_hist[i].rename("mrso").fillna(0.001).sel(lon= slice(260,340))
        X4hist = res_rsds_hist[i].rename("rsds").sel(lon= slice(260,340))
        X5hist = res_vpd_hist[i].rename("vpd").sel(lon= slice(260,340))

        # Standardize predictors
        ensossp = ensossp/ensohist.std(dim = ["time"])
        X1ssp = X1ssp/X1hist.std(dim = "time")
        X2ssp = X2ssp/X2hist.std(dim = "time")
        X3ssp = xr.where((X3ssp/X3hist.std(dim = "time")).fillna(0.001) == np.inf, 0.001, (X3ssp/X3hist.std(dim = "time"))).fillna(0.001)     # sometimes it could happen to have Inf values
        X4ssp = (X4ssp/X4hist.std(dim = "time"))
        X5ssp = (X5ssp/X5hist.std(dim = "time"))

        # Stack on 1D vector
        ensossp = ensossp.stack(cell = ["time"])
        X1ssp = X1ssp.stack(cell = ["lon","lat"])
        X2ssp = X2ssp.stack(cell = ["lon","lat"])
        X3ssp = X3ssp.stack(cell = ["lon","lat"])
        X4ssp = X4ssp.stack(cell = ["lon","lat"])
        X5ssp = X5ssp.stack(cell = ["lon","lat"])

        Y = Y.stack(cell = ["lon","lat"])
        
        # Create Dataarray with ENSO signal in every grid cell to perform regression
        Xenso = xr.DataArray(data=None, coords=[X1ssp.time, X1ssp.cell], dims=["time","cell"])
        for c in Xenso.cell:
            locator = {'cell':c}
            Xenso.loc[locator] = ensossp.values
            Xenso.values = Xenso.values.astype("float64")       # To avoid the resulting values are "object" rather than "float64" 
        Xenso = Xenso.rename("sst")

        # Create empty dataarray to store regression coefficients
        coef = xr.DataArray(data=None, coords=[X1ssp.time,X1ssp.cell], dims=["time","cell"])
        # coef["pr"] =  xr.DataArray(data=None, coords=[X1ssp.cell], dims=["cell"])
        coef["tas"] =  xr.DataArray(data=None, coords=[X1ssp.cell], dims=["cell"])
        coef["mrso"] =  xr.DataArray(data=None, coords=[X1ssp.cell], dims=["cell"])
        coef["sst"] =  xr.DataArray(data=None, coords=[X1ssp.cell], dims=["cell"])
        coef["rsds"] =  xr.DataArray(data=None, coords=[X1ssp.cell], dims=["cell"])
        coef["vpd"] =  xr.DataArray(data=None, coords=[X1ssp.cell], dims=["cell"])

        # Create empty dataarray to store p-values
        pval = xr.DataArray(data=None, coords=[X1ssp.cell], dims=["cell"])
        # pval["pr"] = xr.DataArray(data=None, coords=[X1ssp.cell], dims=["cell"])
        pval["tas"] = xr.DataArray(data=None, coords=[X1ssp.cell], dims=["cell"])
        pval["mrso"] = xr.DataArray(data=None, coords=[X1ssp.cell], dims=["cell"])
        pval["sst"] = xr.DataArray(data=None, coords=[X1ssp.cell], dims=["cell"])
        pval["rsds"] =  xr.DataArray(data=None, coords=[X1ssp.cell], dims=["cell"])
        pval["vpd"] =  xr.DataArray(data=None, coords=[X1ssp.cell], dims=["cell"])

        # Create empty datarray to store R2 and Y predicted 
        r_squared = xr.DataArray(data=None, coords=[X1ssp.cell], dims=["cell"])
        pred = xr.DataArray(data=None, coords=[Y.time, Y.cell], dims=["time","cell"])
        pred["nbp"] = xr.DataArray(data=None, coords=[Y.time, Y.cell], dims=["time", "cell"])

        for c in X1ssp.cell:
            locator = {'cell':c}
            # Check if the mrso value is NaN or 0 for this cell
            if np.isnan(res_mrso_ssp[i].stack(cell = ["lon","lat"]).loc[locator].values).any() or (res_mrso_ssp[i].stack(cell = ["lon","lat"]).loc[locator].values == 0).any():

                # Assign NaN to all result arrays for this cell
                coef["tas"].loc[locator] = np.nan
                coef["mrso"].loc[locator] = np.nan
                coef["rsds"].loc[locator] = np.nan
                coef["vpd"].loc[locator] = np.nan

                pval["tas"].loc[locator] = np.nan
                pval["mrso"].loc[locator] = np.nan
                pval["rsds"].loc[locator] = np.nan
                pval["vpd"].loc[locator] = np.nan

                r_squared.loc[locator] = np.nan
                pred["nbp"].loc[locator] = np.nan
                continue  # Skip this cell and move to the next

            # merge predictors in one dataarray
            df = np.array((X2ssp.loc[locator].values, X3ssp.loc[locator].values, X4ssp.loc[locator].values, X5ssp.loc[locator].values,Xenso.loc[locator].values)); df = df.T
            model = ridge.fit(df,Y.loc[locator].values)
            r2 = ridge.score(df,Y.loc[locator])
            ypred = ridge.predict(df)

            # coef["pr"].loc[locator] = model.coef_[0]
            coef["tas"].loc[locator] = model.coef_[0]
            coef["mrso"].loc[locator] = model.coef_[1]
            coef["rsds"].loc[locator] = model.coef_[2]
            coef["vpd"].loc[locator] = model.coef_[3]
            coef["sst"].loc[locator] = model.coef_[4]

            # p-value of the index "0" refers to the intercept
            # pval["pr"].loc[locator] = stats.coef_pval(model,df,Y.loc[locator].values)[1]
            pval["tas"].loc[locator] = stats.coef_pval(model,df,Y.loc[locator].values)[1]
            pval["mrso"].loc[locator] = stats.coef_pval(model,df,Y.loc[locator].values)[2]
            pval["rsds"].loc[locator] = stats.coef_pval(model,df,Y.loc[locator].values)[3]
            pval["vpd"].loc[locator] = stats.coef_pval(model,df,Y.loc[locator].values)[4]
            pval["sst"].loc[locator] = stats.coef_pval(model,df,Y.loc[locator].values)[5]
            
            r_squared.loc[locator] = r2
            pred["nbp"].loc[locator] = ypred

        coef = coef.unstack()
        coef = xr.concat([coef.tas, coef.mrso, coef.rsds, coef.vpd, coef.sst], dim = "coefficients")
        # coef["pr"] = coef.pr.astype(np.float64)
        coef["tas"] = coef.tas.astype(np.float64)
        coef["mrso"] = coef.mrso.astype(np.float64)
        coef["rsds"] = coef.rsds.astype(np.float64)
        coef["vpd"] = coef.vpd.astype(np.float64)
        coef["sst"] = coef.sst.astype(np.float64)

        pval = pval.unstack()
        pval = xr.concat([pval.tas, pval.mrso, pval.rsds, pval.vpd, pval.sst], dim = "p-values")
        # pval["pr"] = pval.pr.astype(np.float64)
        pval["tas"] = pval.tas.astype(np.float64)
        pval["mrso"] = pval.mrso.astype(np.float64)
        pval["rsds"] = pval.rsds.astype(np.float64)
        pval["vpd"] = pval.vpd.astype(np.float64)
        pval["sst"] = pval.sst.astype(np.float64)
        
        r_squared = r_squared.astype(np.float64).unstack()
        pred = pred.astype(np.float64).unstack()

        coef_ridge_ssp.append(coef)
        r2_ridge_ssp.append(r_squared)
        pred_ridge_ssp.append(pred)
        pval_ridge_ssp.append(pval)
        
    # else:
    #     None
    

# Save and export regression list data
import pickle
data_path = 'G:/My Drive/Amazon_CMIP6/results/'

with open(os.path.join(data_path+"gamma_iav_hist_coef"), "wb") as fp:   #Pickling
    pickle.dump(coef_ridge_hist, fp)
with open(os.path.join(data_path+"gamma_iav_ssp_coef"), "wb") as fp:   #Pickling
    pickle.dump(coef_ridge_ssp, fp)

with open(os.path.join(data_path+"gamma_iav_hist_pval"), "wb") as fp:   #Pickling
    pickle.dump(pval_ridge_hist, fp)
with open(os.path.join(data_path+"gamma_iav_ssp_pval"), "wb") as fp:   #Pickling
    pickle.dump(pval_ridge_ssp, fp)

with open(os.path.join(data_path+"gamma_iav_hist_r2"), "wb") as fp:   #Pickling
    pickle.dump(r2_ridge_hist, fp)
with open(os.path.join(data_path+"gamma_iav_ssp_r2"), "wb") as fp:   #Pickling
    pickle.dump(r2_ridge_ssp, fp)

with open(os.path.join(data_path+"gamma_iav_hist_ypred"), "wb") as fp:   #Pickling
    pickle.dump(pred_ridge_hist, fp)
with open(os.path.join(data_path+"gamma_iav_ssp_ypred"), "wb") as fp:   #Pickling
    pickle.dump(pred_ridge_ssp, fp)



processing historical_ACCESS-ESM1-5
processing historical_ACCESS-ESM1-5
processing historical_ACCESS-ESM1-5
processing historical_ACCESS-ESM1-5
processing historical_ACCESS-ESM1-5
processing historical_CESM2


In [69]:
X5hist.isel(plev = 0)

<xarray.DataArray 'vpd' (time: 165, cell: 1056)>
array([[-0.56341338, -1.13701474, -1.56364053, ..., -0.6286069 ,
        -0.10685729, -0.10181315],
       [ 0.19789259, -0.14881061,  0.24232064, ..., -1.54371648,
        -1.83661314, -1.68742905],
       [-1.52031731, -1.61841485, -1.74482455, ..., -0.9336126 ,
        -0.97444539, -1.00883339],
       ...,
       [-0.85810004, -1.41237936, -1.70740806, ..., -0.2886959 ,
        -0.34611555, -0.58575839],
       [-0.37137885, -0.37733922, -0.79129979, ...,  0.22062479,
         0.40287981,  0.29713288],
       [ 1.47484607,  1.54408747,  0.9454899 , ..., -0.75503888,
        -0.17223527,  0.39756604]])
Coordinates:
    height   float64 2.0
  * time     (time) datetime64[ns] 1849-12-31 1850-12-31 ... 2013-12-31
    plev     float64 9.25e+04
  * cell     (cell) object MultiIndex
  * lon      (cell) float64 260.0 260.0 260.0 260.0 ... 340.0 340.0 340.0 340.0
  * lat      (cell) float64 -29.37 -27.47 -25.58 -23.68 ... 25.58 27.47 29.37

In [120]:
# Open regression list data

import pickle

data_path = 'G:/Shared drives/Amazon_ENSO_work/analysis/'

with open(os.path.join(data_path+"ridge_reg_enso_hist_1901-1960_DJF_coef_c_new"), "rb") as fp:   #Pickling
    coef_ridge_hist = pickle.load(fp)
with open(os.path.join(data_path+"ridge_reg_enso_ssp_2041-2100_DJF_coef_c_new"), "rb") as fp:   #Pickling
    coef_ridge_ssp = pickle.load(fp)

with open(os.path.join(data_path+"ridge_reg_enso_hist_1901-1960_DJF_pval_c_new"), "rb") as fp:   #Pickling
    pval_ridge_hist = pickle.load(fp)
with open(os.path.join(data_path+"ridge_reg_enso_ssp_2041-2100_DJF_pval_c_new"), "rb") as fp:   #Pickling
    pval_ridge_ssp = pickle.load(fp)

with open(os.path.join(data_path+"ridge_reg_enso_hist_1901-1960_DJF_r2_c_new"), "rb") as fp:   #Pickling
    r2_ridge_hist = pickle.load(fp)
with open(os.path.join(data_path+"ridge_reg_enso_ssp_2041-2100_DJF_r2_c_new"), "rb") as fp:   #Pickling
    r2_ridge_ssp = pickle.load(fp)

with open(os.path.join(data_path+"ridge_reg_enso_hist_1901-1960_DJF_ypred_c_new"), "rb") as fp:   #Pickling
    pred_ridge_hist = pickle.load(fp)
with open(os.path.join(data_path+"ridge_reg_enso_ssp_2041-2100_DJF_ypred_c_new"), "rb") as fp:   #Pickling
    pred_ridge_ssp = pickle.load(fp)

### Random Forest Regression

In [ ]:
from sklearn.ensemble import RandomForestRegressor

# stack variables over lon-lat dimensions
# Xenso = nino34_hist[0].rename("sst")#; Xenso = Xenso.stack(cell = ["time"])
X1 = ds_hist_pr[0].rename("pr").sel(lon= slice(260,340))
X2 = ds_hist_tas[0].rename("tas").sel(lon= slice(260,340))
X3 = ds_hist_mrso[0].rename("mrso").sel(lon= slice(260,340), lat = slice(-30,30)).fillna(0)
Y = ds_hist_nbp[0].sel(lon= slice(260,340), lat = slice(-30,30)).fillna(0)

# Standardize predictors
X1 = X1/X1.std(dim = "time")
X2 = X2/X2.std(dim = "time")
X3 = (X3/X3.std(dim = "time")).fillna(0)

# Stack on 1D vector
X1 = X1.stack(cell = ["lon","lat"])
X2 = X2.stack(cell = ["lon","lat"])
X3 = X3.stack(cell = ["lon","lat"])
Y = Y.stack(cell = ["lon","lat"])

# test with ENSO signal in every grid cell
# test = xr.DataArray(data=None, coords=[X1.time, X1.cell], dims=["time","cell"])
# for i in test.cell:
#     locator = {'cell':i}
#     test.loc[locator] = Xenso.values
# test = test.rename("sst")

# merge predictors in one dataarray
X = xr.merge([X1, X2, X3]).to_array().transpose("time", "variable", "cell")

# Define RF parameters
rf_regr = RandomForestRegressor(n_estimators=100,
                              criterion = "squared_error",
                              max_depth=10,
                              max_samples=0.7,
                              random_state=0)

# Create empty dataarray to store regression coefficients
coef = xr.DataArray(data=None, coords=[X1.cell], dims=["cell"])
coef["pr"] =  xr.DataArray(data=None, coords=[X1.cell], dims=["cell"])
coef["tas"] =  xr.DataArray(data=None, coords=[X1.cell], dims=["cell"])
coef["mrso"] =  xr.DataArray(data=None, coords=[X1.cell], dims=["cell"])
# coef["sst"] =  xr.DataArray(data=None, coords=[X1.cell], dims=["cell"])

for i in X.cell:
    locator = {'cell':i}
    model = rf_regr.fit(X.loc[locator],Y.loc[locator])
    coef["pr"].loc[locator] = model.feature_importances_[0]
    coef["tas"].loc[locator] = model.feature_importances_[1]
    coef["mrso"].loc[locator] = model.feature_importances_[2]

coef = coef.unstack()
pr_ridge = coef.pr.astype(np.float64)
tas_ridge = coef.tas.astype(np.float64)
mrso_ridge = coef.mrso.astype(np.float64)
